In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("bigdata2020-sentiment/"))
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/dimitra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['train.csv', 'test_without_labels.csv', 'sample.csv']


# Data Imports

In [ ]:
#importing the training and test data
imdb_data=pd.read_csv('bigdata2020-sentiment/train.csv/train.csv')
test_data=pd.read_csv('bigdata2020-sentiment/test_without_labels.csv/test_without_labels.csv')
print(imdb_data.shape)
imdb_data.head(10)


(25000, 3)


,Content,Label,Id
0,Whoever made this movie must have done it as a...,0,0
1,I love this movie because I grew up around har...,1,1
2,"I had some expectation for the movie, since it...",0,2
3,"During the early 1980's, Kurt Thomas was somet...",0,3
4,Poor Will would be rolling over in his grave i...,0,4
5,After all the crap that Hollywood (and the Ind...,1,5
6,#3 in young John Travolta's trilogy of blockbu...,1,6
7,My first clue about how bad this was going to ...,0,7
8,I really enjoyed the performances of the main ...,1,8
9,Gillian Holroyd (Kim Novak) is a witch. Secret...,1,9


In [ ]:
#Summary of the dataset
#imdb_data.describe()

In [ ]:
#sentiment count
imdb_data['Label'].value_counts()

1    12500
0    12500
Name: Label, dtype: int64

## Data Preprocessing

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['Content']=imdb_data['Content'].apply(denoise_text)
test_data['Content']=test_data['Content'].apply(denoise_text)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['Content']=imdb_data['Content'].apply(remove_special_characters)
test_data['Content']=test_data['Content'].apply(remove_special_characters)

In [ ]:
#Lematizing the text
def simple_lemma(text):
    ps=WordNetLemmatizer()
    text= ' '.join([ps.lemmatize(word) for word in text.split()])
    return text
#Apply function on review column
##imdb_data['Content']=imdb_data['Content'].apply(simple_lemma)
##test_data['Content']=test_data['Content'].apply(simple_lemma)

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
stop.remove('no')
stop.remove('not')

print(stop)
#stop.remove('no')

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token.lower() for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['Content']=imdb_data['Content'].apply(remove_stopwords)
test_data['Content']=test_data['Content'].apply(remove_stopwords)
imdb_data['Content']=imdb_data['Content'].apply(simple_lemma)
test_data['Content']=test_data['Content'].apply(simple_lemma)


{'them', 'by', 'him', 'any', 'this', 'while', 'into', 'mightn', 'other', 'during', "hasn't", 'where', "wasn't", 'shan', 'we', 'because', 'my', 'in', 'under', 'a', 'does', 'was', 'myself', 'herself', 'more', 'his', 'hadn', 'he', "she's", 'theirs', 'how', 'only', 's', 'have', 're', 'having', 'than', 'weren', 'and', 'isn', 'own', 'it', "you'll", 'did', 'were', 'here', 'don', 'been', "it's", 'yours', 'o', 'has', 'couldn', 'an', "couldn't", 'for', 'won', 'as', 'after', 'why', 'yourselves', 'you', 'there', 'their', 'its', 'out', 'few', 'your', 'same', 'hers', 'himself', 'she', 'just', 'between', "didn't", 'off', 'further', 'both', 'll', 'mustn', "needn't", 'ours', 'wasn', 'over', 'now', 'm', "won't", 'with', 'at', 'on', 'will', 'or', "mightn't", 'who', 'some', 'each', 'too', 'yourself', 'below', 'do', 'if', 'aren', 'shouldn', 'then', 'whom', "mustn't", "shouldn't", 'down', 'such', 'ain', 'most', 'all', 'so', 'from', 'hasn', 'haven', 'those', 'should', 'once', 'when', 'needn', "doesn't", "you

In [ ]:
imdb_data.head(10)
test_data.head(10)

,Id,Content
0,0,one best movie seen highly recommend exposure ...
1,1,last month seen lot review italian job many ne...
2,2,movie awful dont even know beginthe positive t...
3,3,worst wrestlemania everthis must see bout many...
4,4,first saw film theater way back 40 kid always ...
5,5,another handheld horror mean another divisive ...
6,6,weird film get impression maker snoozefest spe...
7,7,imdb plot summary erroneously make sound like ...
8,8,without doubt one neil simon best play turned ...
9,9,alice kind movie made 30 40 never attempt even...


In [ ]:
del imdb_data['Id']

In [ ]:
imdb_data.head()

,Content,Label
0,whoever made movie must done joke mean stupide...,0
1,love movie grew around harness racing pat boon...,1
2,expectation movie since nice star cast return ...,0
3,early 1980s kurt thomas something hero united ...,0
4,poor would rolling grave could horiible german...,0


In [ ]:
train = imdb_data.iloc[:23000]
test = imdb_data.iloc[23000:]
real_test = test_data
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(train['Content'])


#transformed test reviews
cv_test_reviews=cv.transform(test['Content'])
real_test_context = cv.transform(real_test['Content'])


print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (23000, 3966181)
BOW_cv_test: (2000, 3966181)


## Multinomial Naive Bayes 

In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train['Label'])
print(mnb_bow)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)

mnb_test_predict = mnb.predict(real_test_context)

[0 0 0 ... 0 1 1]


In [ ]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test['Label'],mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.7475


In [ ]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test['Label'],mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

              precision    recall  f1-score   support

    Positive       0.75      0.77      0.76      1030
    Negative       0.75      0.72      0.74       970

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



## SVM 

In [ ]:
#training the linear svm
svm=SGDClassifier(loss='hinge',random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train['Label'])
print(svm_bow)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [ ]:
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)

svm_test_final = svm.predict(real_test_context)
print(svm_test_final)

[0 1 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [ ]:
svm_bow_score=accuracy_score(test['Label'],svm_bow_predict)
print("svm_bow_score :",svm_bow_score)

svm_bow_score : 0.749


In [ ]:
#Classification report for bag of words 
svm_bow_report=classification_report(test['Label'],svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)

              precision    recall  f1-score   support

    Positive       0.79      0.69      0.74      1030
    Negative       0.71      0.81      0.76       970

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



In [ ]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test['Label'],svm_bow_predict,labels=[1,0])
print(cm_bow)

[[783 187]
 [315 715]]


In [ ]:
prediction = pd.DataFrame(svm_test_final, columns=['Predicted']).to_csv('svm_prediction.csv')

# Keras LSTM model 

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(imdb_data['Content'])
list_tokenized_train = tokenizer.texts_to_sequences(imdb_data['Content'])
maxlen = 500
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = imdb_data['Label']

In [ ]:
embed_size = 300
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 128
epochs = 2

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 300)         3000000   
_________________________________________________________________
bidirectional_15 (Bidirectio (None, None, 64)          85248     
_________________________________________________________________
global_max_pooling1d_15 (Glo (None, 64)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 20)                1300      
_________________________________________________________________
dropout_17 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 20)                420       
_________________________________________________________________
dropout_18 (Dropout)         (None, 20)                0         
__________

In [ ]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.05)

Train on 23750 samples, validate on 1250 samples
Epoch 1/2
23750/23750 [==============================] - 201s 8ms/step - loss: 0.4130 - acc: 0.8206 - val_loss: 0.2848 - val_acc: 0.8784
Epoch 2/2
23750/23750 [==============================] - 195s 8ms/step - loss: 0.2016 - acc: 0.9276 - val_loss: 0.3123 - val_acc: 0.8856


In [ ]:
list_sentences_test = test_data['Content']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
predictions = model.predict(X_te)
y_pred = (predictions > 0.5)

In [ ]:
y_pred


In [ ]:
prediction = pd.DataFrame(y_pred, columns=['Predicted']).to_csv('prediction28.csv')